In [1]:
import json
import os
from tqdm import tqdm

DATA_DIR = '/app/data/aws-rekognition-results'

LABELER = Labeler.objects.get(name='face-identity-rekognition')
print('Labeler:', LABELER.name)

Labeler: face-identity-rekognition


In [2]:
ALL_RESULT_FILES = os.listdir(DATA_DIR)
print('Found: {} results'.format(len(ALL_RESULT_FILES)))

Found: 202972 results


In [3]:
CACHED_IDENTITIES = {}

def ingest_labels(labels):
    if len(labels) == 0:
        return
    face_ids = [l[0] for l in labels]
    existing_qs = FaceIdentity.objects.filter(
        labeler=LABELER, face__id__in=[l[0] for l in labels])
    existing_count = existing_qs.count()
    if existing_count == len(labels):
        return
    else:
        print('Uh oh, some labels missing: expected {}, got {}'.format(
              len(labels), existing_count))
        assert existing_count == 0
    
    new_face_idents = []
    for face_id, ident_name, score in labels:
        ident_name = ident_name.strip()
        if ident_name not in CACHED_IDENTITIES:
            ident, created = Identity.objects.get_or_create(name=ident_name.lower())
            if created:
                print('Created identity:', ident.name)
            CACHED_IDENTITIES[ident_name] = ident
        else:
            ident = CACHED_IDENTITIES[ident_name]
        new_face_idents.append(FaceIdentity(
            labeler=LABELER, face_id=face_id, identity=ident,
            probability=score / 100
        ))
    assert len(FaceIdentity.objects.bulk_create(new_face_idents)) == len(labels)
    created_qs = FaceIdentity.objects.filter(
        labeler=LABELER, face__id__in=[l[0] for l in labels])
#     assert created_qs.count() == len(labels)
    

for fname in tqdm(ALL_RESULT_FILES):
    fpath = os.path.join(DATA_DIR, fname)
    with open(fpath) as f:
        fdata = json.load(f)
    ingest_labels(fdata)

100%|██████████████████████████████████████████████████████| 202972/202972 [39:18<00:00, 86.07it/s]


In [ ]:
FaceIdentity.objects.filter(labeler=LABELER).count()

In [ ]:
small_count = 0
for fname in ALL_RESULT_FILES:
    fpath = os.path.join(DATA_DIR, fname)
    with open(fpath) as f:
        fdata = json.load(f)
    if len(fdata) == 0:
        print(fpath)
        small_count += 1
small_count